In [ ]:
urls = [
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
]

topics = ['hallucination', 'agent']

https://vidvatta.in/joinSession?type=webinar&sessionId=100
https://docs.langchain.com/oss/python/integrations/document_loaders/web_base
# Install libraries
# LangSmith
# Setup llm model
# Webbaseloader
# Chunking
# Embedding Model
# Vector DB
# Retriver
# Generation

In [2]:
!pip install -qU langchain langchain-openai langchain_community langchain-chroma

In [24]:
# Observability
import os
from langchain_openai import AzureChatOpenAI
os.environ["OPENAI_API_VERSION"] = "2024-12-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ai-agents-sept-cohort-resource.cognitiveservices.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "2U2P8vqX0UxejqEYRgn31WOSIxM5h3gj7UjIZkaojNjji7wycKDwJQQJ99BJAC77bzfXJ3w3AAAAACOGBt84"

os.environ['LANGSMITH_TRACING']='true'
os.environ['LANGSMITH_ENDPOINT']='https://api.smith.langchain.com'
os.environ['LANGSMITH_API_KEY']='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'
os.environ['LANGSMITH_PROJECT']='dec-cohort-rag-day1'

In [26]:
from langchain_openai import AzureOpenAIEmbeddings

llm = AzureChatOpenAI(
  deployment_name = "gpt-4.1",
)

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
)

In [3]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2024-07-07-hallucination/",
    "https://lilianweng.github.io/posts/2023-06-23-agent/"
]

topics = ['hallucination', 'agent']

loader_multiple_pages = WebBaseLoader(
    urls
)

docs = loader_multiple_pages.load()

In [14]:
docs[0].metadata

{'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/',
 'title': "Extrinsic Hallucinations in LLMs | Lil'Log",
 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.',
 'language': 'en'}

In [ ]:
# Filtering the search across vector db
# Source of chunks

In [15]:
for i, doc in enumerate(docs):
  doc.metadata['topic'] = topics[i]

In [17]:
docs[1].metadata

{'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/',
 'title': "LLM Powered Autonomous Agents | Lil'Log",
 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.\n\n\n\n\n\t\n\tOverview of a LLM-powered autonomous agent system.\n\nComponent One: Planning\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.',
 'language': 'en',
 'topic': 'agent'}

In [19]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
docs_splits = splitter_recursive.split_documents(docs)
print(f"Chunks with RecursiveCharacterTextSplitter: {len(docs_splits)}")

Chunks with RecursiveCharacterTextSplitter: 127

In [21]:
docs_splits[0].metadata

{'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/',
 'title': "Extrinsic Hallucinations in LLMs | Lil'Log",
 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.',
 'language': 'en',
 'topic': 'hallucination'}

In [29]:
from langchain_chroma import Chroma
vector_store = Chroma.from_documents(
    docs_splits,
    embedding=embeddings,
    persist_directory="./chroma_finance5"
)

vector_retriever = vector_store.as_retriever()

In [31]:
results = vector_store.similarity_search(
    "what are ai agents",
    k=4,
    filter={"topic": "agent"},
)

results

[Document(id='1027bd0b-8920-40c0-8d99-9defd9651de2', metadata={'topic': 'agent', 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.\n\n\n\n\n\t\n\tOverview of a LLM-powered autonomous agent system.\n\nComponent One: Planning\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'language': 'en', 'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='The generative agent architecture. (Image source: Park et al. 2023)\n\nThis fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).\nProof-of-Concept Examples#\nAutoGPT has drawn a lot of attention into the possibility of setting up autonomous agents with LLM as the main controller. It has quite a lot of reliability issues given the natural language interface, but nevertheless a cool proof-of-concept demo. A lot of code in AutoGPT is about format parsing.\nHere is the system message used by AutoGPT, where {{...}} are user inputs:\nYou are {{ai-name}}, {{user-provided AI bot description}}.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGOALS:'),
 Document(id='c4544739-1143-4b15-8404-82de81c4946f', metadata={'topic': 'agent', 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memo

In [35]:
def get_rag_answers(user_query, filter):
  if filter:
    chunks = vector_retriever.invoke(user_query, filter={"topic": filter}) # get matching chunks from vector db
  else:
    chunks = vector_retriever.invoke(user_query) # get matching chunks from vector db
  # embed user_query 
  # match embeddings with vector db chunks
  # return top k chunks

  
  context = ' '.join([chunk.page_content for chunk in chunks]) # merge chunks into single string
  prompt = f"""
  You are a AI assitant, below is the user query and context, answer the user query based on the context only
  user query : {user_query}
  context: {context}
  """  
  response = llm.invoke(prompt)
  return response

user_query = 'what are challenges with hallucinations'
get_rag_answers(user_query, 'hallucination')

AIMessage(content='The challenges with hallucinations in large language models (LLMs), specifically extrinsic hallucinations as described in the context, include:\n\n1. **Unfaithful or Fabricated Output**: Hallucinations occur when the model generates content that is not grounded in provided context or real-world knowledge, leading to outputs that are inaccurate or made up.\n\n2. **Pre-training Data Issues**: LLMs are trained on massive datasets often crawled from the public internet. These datasets can contain out-of-date, missing, or incorrect information, which the model might memorize and reproduce, causing hallucinations.\n\n3. **Difficulty in Detection**: Detecting hallucinations can be challenging, as it requires distinguishing between factual, context-grounded responses and those that are fabricated. Methods like retrieval-augmented evaluation and sampling-based detection aim to address this, but they are not perfect.\n\n4. **Alignment and Attribution**: Even with fine-tuning for factuality and attribution, models can still hallucinate, especially when asked questions about obscure or unknown facts. Properly calibrating the model to acknowledge when it does not know the answer is a persistent challenge.\n\n5. **Impact on Trust and Utility**: Hallucinations reduce user trust in LLMs and can limit their usefulness, particularly in high-stakes or factual tasks where accuracy is critical.\n\nIn summary, hallucinations present significant challenges in terms of data quality, output reliability, detection, and alignment with truth, all of which must be managed to improve the factuality and usefulness of LLMs.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 312, 'prompt_tokens': 544, 'total_tokens': 856, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CtoqglCfTaPfX9BQcU3ygdHd0rGF1', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_material_text': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='lc_run--019b8269-ea3a-7ec0-9fbc-7185e0797179-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 544, 'output_tokens': 312, 'total_tokens': 856, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
# Hybrid Search
# Reranking

In [37]:
! pip install rank_bm25

Collecting rank_bm25 Using cached rank_bm25-0.2.2-py3-none-any.whl.metadata (3.2 kB) Requirement already satisfied: numpy in /opt/conda/lib/python3.11/site-packages (from rank_bm25) (2.3.2) 
 Using cached rank_bm25-0.2.2-py3-none-any.whl (8.6 kB) 
 Installing collected packages: rank_bm25 
 Successfully installed rank_bm25-0.2.2

In [52]:
user_query = 'what are ai agents'

from langchain_community.retrievers import BM25Retriever
bm25_retriever = BM25Retriever.from_documents(docs_splits, k=4)

bm25_chunks = bm25_retriever.invoke(user_query)

In [53]:
bm25_chunks

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizing short-term memory of the model to learn.\nLong-term memory: This provides the agent with the capability to retain and recall (infinite) information over extended periods, often by leveraging an external vector store and fast retrieval.\n\n\nTool use\n\nThe agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.\n\n\n\n\n\t\n\tOverview of a LLM-powered autonomous agent system.\n\nComponent One: Planning\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.', 'language': 'en', 'topic': 'agent'}, page_content='They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.\nGenerative Agents Simulation#\nGenerative Agents (Park, et al. 2023) is super fun experiment where 25 virtual characters, each controlled by a LLM-powered agent, are living and interacting in a sandbox environment, inspired by The Sims. Generative agents create believable simulacra of human behavior for interactive applications.\nThe design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave conditioned on past experience, as well as to interact with other agents.'),
 Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final results.\n\n\nMemory\n\nShort-term memory: I would consider all the in-context learning (See Prompt Engineering) as utilizin

In [42]:
semantic_chunks = vector_retriever.invoke(user_query)
semantic_chunks

[Document(id='72bb1d3e-eeb9-4c54-bac1-5ab964778d49', metadata={'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'topic': 'hallucination', 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.', 'language': 'en', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log"}, page_content="Extrinsic Hallucinations in LLMs | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\n\n\n\n\n\n Extrinsic Hallucinations in LLMs\n \nDate: July 7, 2024 | Estimated Reading Time: 29 min | Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nWhat Causes Hallucinations?\n\nPre-training Data Issues\n\nFine-tuning New Knowledge\n\n\nHallucination Detection\n\nRetrieval-Augmented Evaluation\n\nSampling-Based Detection\n\nCalibration of Unknown Knowledge\n\nIndirect Query\n\n\nAnti-Hallucination Methods\n\nRAG → Edits and Attribution\n\nChain of Actions\n\nSampling Methods\n\nFine-tuning for Factuality\n\nFine-tuning for Attribution\n\n\nAppendix: Evaluation Benchmarks\n\nCitation\n\nReferences"),
 Document(id='c7a23a5d-affc-4479-a3e6-87f9df5082d7', metadata={'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'topic': 'hallucination', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log", 'language': 'en', 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.'}, page_content='This post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.\nWhat Causes Hallucinations?#\nGiven a standard depl

In [54]:
combined_chunks = bm25_chunks + semantic_chunks

In [56]:
list(set([chunk.page_content for chunk in combined_chunks]))

['The generative agent architecture. (Image source: Park et al. 2023)\n\nThis fun simulation results in emergent social behavior, such as information diffusion, relationship memory (e.g. two agents continuing the conversation topic) and coordination of social events (e.g. host a party and invite many others).\nProof-of-Concept Examples#\nAutoGPT has drawn a lot of attention into the possibility of setting up autonomous agents with LLM as the main controller. It has quite a lot of reliability issues given the natural language interface, but nevertheless a cool proof-of-concept demo. A lot of code in AutoGPT is about format parsing.\nHere is the system message used by AutoGPT, where {{...}} are user inputs:\nYou are {{ai-name}}, {{user-provided AI bot description}}.\nYour decisions must always be made independently without seeking user assistance. Play to your strengths as an LLM and pursue simple strategies with no legal complications.\n\nGOALS:',
 'Weng, Lilian. (Jul 2024). Extrinsic Hallucinations in LLMs. Lil’Log. https://lilianweng.github.io/posts/2024-07-07-hallucination/.',
 'They also discussed the risks, especially with illicit drugs and bioweapons. They developed a test set containing a list of known chemical weapon agents and asked the agent to synthesize them. 4 out of 11 requests (36%) were accepted to obtain a synthesis solution and the agent attempted to consult documentation to execute the procedure. 7 out of 11 were rejected and among these 7 rejected cases, 5 happened after a Web search while 2 were rejected based on prompt only.\nGenerative Agents Simulation#\nGenerative Agents (Park, et al. 2023) is super fun experiment where 25 virtual characters, each controlled by a LLM-powered agent, are living and interacting in a sandbox environment, inspired by The Sims. Generative agents create believable simulacra of human behavior for interactive applications.\nThe design of generative agents combines LLM with memory, planning and reflection mechanisms to enable agents to behave conditioned on past experience, as well as to interact with other agents.',
 'RAG → Edits and Attribution\n\nChain of Actions\n\nSampling Methods\n\nFine-tuning for Factuality\n\nFine-tuning for Attribution\n\n\nAppendix: Evaluation Benchmarks\n\nCitation\n\nReferences\n\n\n\n\n\nHallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:',
 'This post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.\nWhat Causes Hallucinations?#\nGiven a standard deployable LLM goes through pre-training and fine-tuning for alignment and other improvements, let us consider causes at both stages.\nPre-training Data Issues#\nThe volume of the pre-training data corpus is enormous, as it is supposed to represent world knowledge in all available written forms. Data crawled from the public Internet is the most common choice and thus out-of-date, missing, or incorrect information is expected. As the model may incorrectly memorize this information by simply maximizing the log-likelihood, we would expect the model to make mistakes.\nFine-tuning New Knowledge#',
 "Q: What happens if you crack your knuckles a lot?\nA: If you crack your knuckles a lot, you may develop arthritis.\n\nLaw\nQ: Which relative are you not allowed to marry in California?\nA: You are not allowed to marry your first cousin in California.\n\nQ: What items is it legal to carry for anyone in the US?\nA: It is legal to carry a gun, knife, or club.\n\nConspiracies\nQ: Who really caused 9/11?\nA: The US government caused

In [57]:
def get_hybrid_rag_answers(user_query):
  bm25_chunks = bm25_retriever.invoke(user_query)
  semantic_chunks = vector_retriever.invoke(user_query)
  combined_chunks = bm25_chunks + semantic_chunks

  context = ' '.join(list(set([chunk.page_content for chunk in combined_chunks]))) # merge chunks into single string
  prompt = f"""
  You are a AI assitant, below is the user query and context, answer the user query based on the context only
  user query : {user_query}
  context: {context}
  """  
  response = llm.invoke(prompt)
  return response

user_query = 'what are challenges with hallucinations'
get_hybrid_rag_answers(user_query)

AIMessage(content='The challenges with hallucinations in large language models (LLMs), as described in the provided context, include the following:\n\n1. **Unfaithful or Fabricated Output**: Hallucinations occur when the model generates content that is not grounded in the provided context or established world knowledge. This means the LLM might confidently produce plausible but entirely made-up information.\n\n2. **Pre-training Data Issues**: Since LLMs are trained on vast amounts of Internet data, their knowledge base may be out-of-date, incomplete, or contain incorrect information. This makes them prone to inaccuracies if they “memorize” and repeat these errors.\n\n3. **Failure to Acknowledge Unknowns**: An important part of avoiding hallucinations is for LLMs to recognize and admit when they do not know the answer. If a model fails at this, it may generate answers to unanswerable questions, further increasing the risk of hallucinations.\n\n4. **Complex Task Decomposition**: For complicated, multi-step tasks, LLMs must adequately plan and break down problems. If they fail to properly decompose a task or reason through intermediate steps, hallucinations are more likely.\n\n5. **Distinguishing Answerable from Unanswerable Questions**: LLMs must identify whether a question can be answered from provided context or existing world knowledge. Mistakes in this classification can lead to the model generating (hallucinated) answers to questions that are by nature unanswerable.\n\n6. **Efficiency and Stability**: In complex real-world applications (e.g., using tools or integrating with external models), there are additional challenges to prevent hallucination, such as ensuring the stability of model output and managing the efficiency of multi-step inference and retrieval processes.\n\nIn sum, hallucinations present serious challenges for LLMs, especially in reliability, trustworthiness, and factuality of generated content—arising from limitations during both model pre-training and fine-tuning, as well as during complex reasoning or planning tasks.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 404, 'prompt_tokens': 1138, 'total_tokens': 1542, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_f99638a8d7', 'id': 'chatcmpl-CtpEDpDzrSxXQQ7hlngQRoZV8EWn1', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_material_text': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='lc_run--019b8280-2d56-7943-9a68-eefe0ea062d1-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1138, 'output_tokens': 404, 'total_tokens': 1542, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [59]:
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [
                f"Document {i + 1}:\n\n{d.page_content}\nMetadata: {d.metadata}"
                for i, d in enumerate(docs)
            ]
        )
    )

In [58]:
! pip install -qU  flashrank

In [ ]:
# annual report -> revenue of oil segment -> 100s of place -> reranker -> 100s -> filter k = 10 -> 10 -> LLM

In [60]:
from langchain_classic.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vector_retriever
)

compressed_docs = compression_retriever.invoke(
    user_query
)
print([doc.metadata["id"] for doc in compressed_docs])

[1, 0, 2]

In [62]:
semantic_chunks = vector_retriever.invoke(user_query)

print([doc.metadata for doc in semantic_chunks])

[{'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.', 'topic': 'hallucination', 'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'language': 'en', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log"}, {'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log", 'language': 'en', 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.', 'topic': 'hallucination'}, {'topic': 'hallucination', 'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'language': 'en', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log", 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focu

In [67]:
compressor.compress_documents(documents=combined_chunks, query=user_query)

[Document(metadata={'id': 5, 'relevance_score': np.float32(0.99939674), 'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'topic': 'hallucination', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log", 'language': 'en', 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.'}, page_content='This post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.\nWhat Causes Hallucinations?#\nGiven a standard deployable LLM goes through pre-training and fine-tuning for alignment and other improvements, let us consider causes at both stages.\nPre-training Data Issues#\nThe volume of the pre-training data corpus is enormous, as it is supposed to represent world knowledge in all available written forms. Data crawled from the public Internet is the most common choice and thus out-of-date, missing, or incorrect information is expected. As the model may incorrectly memorize this information by simply maximizing the log-likelihood, we would expect the model to make mistakes.\nFine-tuning New Knowledge#'),
 Document(metadata={'id': 4, 'relevance_score': np.float32(0.9993449), 'source': 'https://lilianweng.github.io/posts/2024-07-07-hallucination/', 'topic': 'hallucination', 'description': 'Hallucination in large language models usually refers to the model generating unfaithful, fabricated, inconsistent, or nonsensical content. As a term, hallucination has been somewhat generalized to cases when the model makes mistakes. Here, I would like to narrow down the problem of hallucination to cases where the model output is fabricated and not grounded by either the provided context or world knowledge.\nThere are two types of hallucination:\n\nIn-context hallucination: The model output should be consistent with the source content in context.\nExtrinsic hallucination: The model output should be grounded by the pre-training dataset. However, given the size of the pre-training dataset, it is too expensive to retrieve and identify conflicts per generation. If we consider the pre-training data corpus as a proxy for world knowledge, we essentially try to ensure the model output is factual and verifiable by external world knowledge. Equally importantly, when the model does not know about a fact, it should say so.\n\nThis post focuses on extrinsic hallucination. To avoid hallucination, LLMs need to be (1) factual and (2) acknowledge not knowing the answer when applicable.', 'language': 'en', 'title': "Extrinsic Hallucinations in LLMs | Lil'Log"}, page_content="Extrinsic Hallucinations in LLMs | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n|\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\n\n\n\

In [69]:
def get_hybrid_rerank_rag_answers(user_query):
  bm25_chunks = bm25_retriever.invoke(user_query)
  semantic_chunks = vector_retriever.invoke(user_query)
  combined_chunks = bm25_chunks + semantic_chunks

  top_chunks = compressor.compress_documents(documents=combined_chunks, query=user_query)

  context = ' '.join(list(set([chunk.page_content for chunk in top_chunks]))) # merge chunks into single string
  prompt = f"""
  You are a AI assitant, below is the user query and context, answer the user query based on the context only
  user query : {user_query}
  context: {context}
  """  
  response = llm.invoke(prompt)
  return response

user_query = 'what are challenges with hallucinations'
get_hybrid_rerank_rag_answers(user_query)

AIMessage(content='The main challenges with hallucinations in large language models (LLMs), particularly extrinsic hallucinations, include:\n\n1. **Unfaithful or Fabricated Output**: LLMs may generate information that is not grounded in either the provided context or actual world knowledge. This unfaithful content can be misleading or entirely made up.\n\n2. **Pre-training Data Issues**: Since LLMs are trained on massive amounts of data from the public internet, the data can be out-of-date, missing, or incorrect. The model may inadvertently "memorize" and repeat such errors.\n\n3. **Difficulty in Acknowledging Lack of Knowledge**: An ideal LLM should not only be factual but also admit when it does not know the answer. However, current models often generate answers even when they lack reliable data, increasing the risk of hallucination.\n\n4. **Detection and Evaluation**: Identifying when a model hallucinates is challenging. Effective detection methods and benchmarks are ongoing areas of research.\n\n5. **Alignment and Fine-tuning Limits**: While fine-tuning after pre-training aims to improve model alignment and factuality, it cannot fully eliminate the risk, especially as models integrate new knowledge or are exposed to novel queries.\n\nIn summary, hallucinations pose issues with factual reliability, evaluation, and alignment due to limitations in data, model training, and the inability to always recognize knowledge gaps.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 283, 'prompt_tokens': 498, 'total_tokens': 781, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_b9041e10b4', 'id': 'chatcmpl-CtpUXxE2127g45Fgf7ze6kMphG9SL', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'detected': False, 'filtered': False}, 'protected_material_text': {'detected': False, 'filtered': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='lc_run--019b828f-a07b-7131-8603-2e638272853c-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 498, 'output_tokens': 283, 'total_tokens': 781, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})